# 🛰️ BIOMASS L1C Product Analysis and Visualization 🌳

(c) European Space Agency (ESA) Licensed under ESA Software Community Licence Permissive (Type 3) – v2.4

# Execution and scope
The following libraries are needed to execute this notebook: rasterio (gdal), netCDF4, numpy, scipy, matplotlib, xml. Environment files tested on win/linux machines can be provided upon request (just send an email to muriel.pinheiro@esa.int) 
“"(c) European Space Agency (ESA) - Licensed under  ESA Software Community Licence Permissive (Type 3) – v2.4"”

#### The aim of the notebook
This notebook is intended to demonstrate the processing and visualization of BIOMASS L1C products, including:
Section 1 : Reading and visualizing COG-formatted SAR measurement data (amplitude and phase)
Section 2 : Extracting and displaying LUT (Look-Up Table) variables from the annotated NetCDF datasets
Section 3 : Upsampling LUT variables to match the full-resolution SLC grid
Section 4 : Computing interferograms and coherence maps

#### Replication of notebook
This notebook can be replicated/executed starting from the shared TDS. For further details, you can consult the shared documentation or simply send an email to muriel.pinheiro@esa.int

## 📚 Index

- [🌿 Section 1: BIOMASS L1C – Amplitude and Phase (COG) Visualization](#section-1-biomass-l1c--amplitude-and-phase-cog-visualization)
- [🌱 Section 2: Reading and Exploring LUT Variables (NetCDF)](#section-2-reading-and-exploring-lut-variables-netcdf)
- [🍃 Section 3: Upsampling LUTs onto the SLC Grid](#section-3-upsampling-luts-onto-the-slc-grid)
- [🌲 Section 4: Basic Interferometric Products (Interferogram and Coherence)](#section-4-basic-interferometric-products-interferogram-and-coherence)


## 🌿 Section 1: BIOMASS L1C – Amplitude and Phase (COG) Visualization

In this section we:
- Access to measurement (COG - Cloud Optimized GeoTIFF)files containing amplitude and phase.
- Reconstruct the complex SLC images for each polarization (HH, XX, VV).
- Visualize amplitude images in decibel (dB) scale.

In [ ]:
%matplotlib inline

In [ ]:
import rasterio
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
import BiomassProduct 
import warnings
from pathlib import Path

# Suppress deprecation and other warnings for cleaner output
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore')

# Define the folder containing the Level 1c products
#folder_TDS = Path("..") / "WD_F158_1C"
folder_TDS = r"WD_F158_1C"
# Load the STA product group from the specified folder
loader = BiomassProduct.STAProductGroupLoader(folder_TDS)

# Print a summary of the products found (names and paths)
loader.print_summary()
print (loader)
# Access the primary and secondary products by their assigned names
product_primary = loader.get_instance_by_name("product_primary")
product_secondary1 = loader.get_instance_by_name("product_secondary1")

# Access and print the noDataValue from the primary product's annotation
noDataValue=product_primary.noDataValue
print("NoData Value (primary product):", noDataValue)

# Loop over all products and print their names and paths
for prod in loader.products:
    print(f"{prod['name']}: loaded from {prod['path']}")

# Print available LUT variable names for the primary product
print("\nAvailable LUT variables:")
print(product_primary.lut_variables.keys())

# Access and print the 'geometry_terrainSlope' LUT variable as an example
print("\nExample LUT variable: 'geometry_terrainSlope'")
print(product_primary.lut_variables['geometry_terrainSlope'])

In [ ]:
# Retrieve the full path to the amplitude and phase measurement files
path_amp = product_secondary1.measurement_abs_file
path_phase =product_secondary1.measurement_phase_file

# Print the paths to verify they are correctly retrieved
print(path_amp)
print(path_phase)

# Note: Channels in the measurement files are stored in lexicographic order.
# For example: (HH, HV, VH, VV) or (HH, XX, VV) for inputs compatible with L2A processing.

## plot the amp and phases: get channels as nupy.ndarray objects --> 
#this is just one L1C (data) data the same came be reiterated for the other data 

# Define no-data value
nan_value = noDataValue

# Read amplitude bands
with rasterio.open(path_amp) as amp_raster: #get the amp channels
    amp_hh = np.ma.masked_equal(amp_raster.read(1), nan_value)
    amp_xx = np.ma.masked_equal(amp_raster.read(2), nan_value)
    amp_vv = np.ma.masked_equal(amp_raster.read(3), nan_value)

# Read phase bands
with rasterio.open(path_phase) as phase_raster: #get the phase channels
    phase_hh = np.ma.masked_equal(phase_raster.read(1), nan_value)
    phase_xx = np.ma.masked_equal(phase_raster.read(2), nan_value)
    phase_vv = np.ma.masked_equal(phase_raster.read(3), nan_value)

# Check mask consistency
assert np.all(amp_hh.mask == phase_hh.mask)
assert np.all(amp_xx.mask == phase_xx.mask)
assert np.all(amp_vv.mask == phase_vv.mask)

#print(f'phase_** az is {phase_hh.shape[0]} rg {phase_hh.shape[1]}')

# Reconstruct the complex SLCs
hh_slc = amp_hh * np.exp(1j * phase_hh)
xx_slc = amp_xx * np.exp(1j * phase_xx)
vv_slc = amp_vv * np.exp(1j * phase_vv)


#### Plot amplitude images (dB) for HH, XX, VV

In [ ]:
# Convert to dB scale
hh_db = 20 * np.log10(amp_hh)
xx_db = 20 * np.log10(amp_xx)
vv_db = 20 * np.log10(amp_vv)

#get the scalebar limits

hh_db_avg = np.mean(hh_db[2000:25000, :])
hh_db_std = np.std(hh_db[2000:25000, :])


xx_db_avg = np.mean(xx_db[2000:25000, :])
xx_db_std = np.std(xx_db[2000:25000, :])


vv_db_avg = np.mean(vv_db[2000:25000, :])
vv_db_std = np.std(vv_db[2000:25000, :])
## plot the amp and phases: plot the data

std_multiplier = 3
aspect_ratio = 'auto' #0.1
colorbar_fraction = 0.05
colorbar_pad = 0.04

plt.close('all')
fig, axes = plt.subplots(1,3) # adjust plots scale and plot size if possibile

cax1 = axes[0].imshow( 
    hh_db, 
    cmap='rainbow',
    aspect= aspect_ratio , 
    vmin = hh_db_avg - std_multiplier * hh_db_std, 
    vmax = hh_db_avg + std_multiplier * hh_db_std ) 

axes[0].set_title('db HH')

cax2 = axes[1].imshow( 
    xx_db, 
    cmap='rainbow',
    aspect= aspect_ratio , 
    vmin = xx_db_avg - std_multiplier * xx_db_std, 
    vmax = xx_db_avg + std_multiplier * xx_db_std )
axes[1].set_title('db XX')
axes[1].yaxis.set_tick_params(labelleft=False)

cax3 = axes[2].imshow( 
    vv_db, 
    cmap='rainbow',
    aspect= aspect_ratio , 
    vmin = vv_db_avg - std_multiplier * vv_db_std, 
    vmax = vv_db_avg + std_multiplier * vv_db_std )
axes[2].set_title('db VV')
axes[2].yaxis.set_tick_params(labelleft=False)

fig.colorbar(cax1, ax=axes[0], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad) 
fig.colorbar(cax2, ax=axes[1], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.colorbar(cax3, ax=axes[2], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.tight_layout()
plt.show()

## 🌱 Section 2: Reading and Exploring LUT Variables (NetCDF)
In this section we:
- Access the Look-Up Table (LUT) dataset provided in NetCDF format.
- Extract key variables grouped under `radiometry`,`denoising`, `geometry`, `coregistration`, `skpPhaseCalibration`, `baselineAndIonosphereCorrection`.
- Visualize selected LUT components (e.g., flattening phase screen).

#### Section 2.1: Print LUT radiometry group 
This includes:
1. sigmaNought
2. gammaNought

In [ ]:
import rasterio
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
import BiomassProduct 
import warnings
from pathlib import Path

# Suppress deprecation and other warnings for cleaner output
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore')

# Define the folder containing the Level 1c products
#folder_TDS = Path("..") / "L1c"
folder_TDS =r"WD_F158_1C"
# Load the STA product group from the specified folder
loader = BiomassProduct.STAProductGroupLoader(folder_TDS)

# Print a summary of the products found (names and paths)
loader.print_summary()

# Access the primary and secondary products by their assigned names
product_primary = loader.get_instance_by_name("product_primary")
product_secondary = loader.get_instance_by_name("product_secondary1")

# Access and print the noDataValue from the primary product's annotation
noDataValue=product_primary.noDataValue
print("NoData Value (primary product):", noDataValue)


In [ ]:
# Check if radiometry LUT is loaded
if hasattr(product_secondary, "radiometry_sigmaNought") and hasattr(product_secondary, "radiometry_gammaNought"):

    sigmaNought = product_secondary.radiometry_sigmaNought
    gammaNought = product_secondary.radiometry_gammaNought

    # Compute stats
    sigmaNought_avg = np.mean(sigmaNought)
    sigmaNought_std = np.std(sigmaNought)
    gammaNought_avg = np.mean(gammaNought)
    gammaNought_std = np.std(gammaNought)

    # Plot settings
    std_multiplier = 3
    aspect_ratio ='auto' # 0.5
    colorbar_fraction = 0.05
    colorbar_pad = 0.04

    plt.close('all')
    fig, axes = plt.subplots(1,2) 
    ax1 = axes[0].imshow(
        sigmaNought, 
        cmap='rainbow',
        aspect=aspect_ratio, 
        vmin =  sigmaNought_avg - std_multiplier * sigmaNought_std,
        vmax =  sigmaNought_avg + std_multiplier * sigmaNought_std
    )
    axes[0].set_title('Sigma factor')


    ax2 = axes[1].imshow(
        gammaNought, 
        cmap='rainbow',
        aspect=aspect_ratio,
        vmin = gammaNought_avg - std_multiplier *gammaNought_std,
        vmax = gammaNought_avg + std_multiplier *gammaNought_std
    )
    axes[1].set_title('Gamma factor')
    axes[1].yaxis.set_tick_params(labelleft=False)

    fig.colorbar(ax1, ax=axes[0], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
    fig.colorbar(ax2, ax=axes[1], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
    fig.tight_layout()
    plt.show()
else:
    print("❌ Radiometry LUT or required variables not found in product.")

#### Section 2.2: Print LUT denoising group 
This includes (Requires an upsampling before its application on the original COG temporal grid):
1. denoisingHH
2. denoisingXX
3. denoisingVV

In [ ]:
if hasattr(product_secondary1, "denoising_denoisingHH") and hasattr(product_secondary1, "denoising_denoisingXX") and hasattr(product_secondary1, "denoising_denoisingVV"):


    # Get denoising variables directly from the product
    denoisingHH = product_secondary.denoising_denoisingHH
    denoisingXX = product_secondary.denoising_denoisingXX
    denoisingVV = product_secondary.denoising_denoisingVV


    # plot the lut denoising denoisingHH, denoisingXX, denoisingVV
    denoisingHH_avg = np.mean(denoisingHH)
    denoisingHH_std = np.std(denoisingHH)

    denoisingXX_avg = np.mean(denoisingXX)
    denoisingXX_std = np.std(denoisingXX)


    denoisingVV_avg = np.mean(denoisingVV)
    denoisingVV_std = np.std(denoisingVV)

    std_multiplier = 3
    aspect_ratio = 'auto' #0.5
    colorbar_fraction = 0.05
    colorbar_pad = 0.04

    plt.close('all')
    fig, axes = plt.subplots(1,3) 
    ax1 = axes[0].imshow( 
        denoisingHH, 
        cmap='rainbow',
        aspect=aspect_ratio,
        vmin =  denoisingHH_avg - std_multiplier * denoisingHH_std,
        vmax =  denoisingHH_avg + std_multiplier * denoisingHH_std
    )
    axes[0].set_title('denoisingXX')

    ax2 = axes[1].imshow( 
        denoisingXX, 
        cmap='rainbow',
        aspect=aspect_ratio,
        vmin =  denoisingXX_avg - std_multiplier * denoisingXX_std,
        vmax =  denoisingXX_avg + std_multiplier * denoisingXX_std
    )
    axes[1].set_title('denoisingXX')
    axes[1].yaxis.set_tick_params(labelleft=False)

    ax3 = axes[2].imshow( 
        denoisingVV, 
        cmap='rainbow',
        aspect=aspect_ratio,
        vmin =  denoisingVV_avg - std_multiplier * denoisingVV_std,
        vmax =  denoisingVV_avg + std_multiplier * denoisingVV_std
    )
    axes[2].set_title('denoisingVV')
    axes[2].yaxis.set_tick_params(labelleft=False)


    fig.colorbar(ax1, ax=axes[0], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
    fig.colorbar(ax2, ax=axes[1], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
    fig.colorbar(ax3, ax=axes[2], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
    fig.tight_layout()
    plt.show()
else:
    print("❌ Denoising LUT or required variables not found in product.")    

#### Section 2.3: Print LUT geometry 

This includes (Requires an upsampling before its application on the original COG temporal grid):
1. latitude
2. longitude
3. height
4. incidenceAngle
5. elevationAngle
6. terrainSlope

In [ ]:
#plot the geometry group data
std_multiplier = 3
aspect_ratio = 'auto' #0.5
colorbar_fraction = 0.05
colorbar_pad = 0.04

plt.close('all')
fig, axes = plt.subplots(2, 3, figsize=(12, 8))

# Plot latitude
if hasattr(product_secondary, "geometry_latitude"):
    latitude = product_secondary1.geometry_latitude
    latitude_avg = np.mean(latitude)
    latitude_std = np.std(latitude)
    ax1 = axes[0, 0].imshow(latitude, cmap='rainbow', aspect=aspect_ratio,
        vmin=latitude_avg - std_multiplier * latitude_std,
        vmax=latitude_avg + std_multiplier * latitude_std)
    axes[0, 0].set_title("latitude [deg]")
    fig.colorbar(ax1, ax=axes[0, 0], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
else:
    axes[0, 0].set_title("latitude [deg]\n[not available]")
    axes[0, 0].axis('off')

# Plot longitude
if hasattr(product_secondary, "geometry_longitude"):
    longitude = product_secondary1.geometry_longitude
    longitude_avg = np.mean(longitude)
    longitude_std = np.std(longitude)
    ax2 = axes[0, 1].imshow(longitude, cmap='rainbow', aspect=aspect_ratio,
        vmin=longitude_avg - std_multiplier * longitude_std,
        vmax=longitude_avg + std_multiplier * longitude_std)
    axes[0, 1].set_title("longitude [deg]")
    axes[0, 1].yaxis.set_tick_params(labelleft=False)
    fig.colorbar(ax2, ax=axes[0, 1], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
else:
    axes[0, 1].set_title("longitude [deg]\n[not available]")
    axes[0, 1].axis('off')

# Plot height
if hasattr(product_secondary, "geometry_height"):
    height = product_secondary1.geometry_height
    height_avg = np.mean(height)
    height_std = np.std(height)
    ax3 = axes[0, 2].imshow(height, cmap='rainbow', aspect=aspect_ratio,
        vmin=height_avg - std_multiplier * height_std,
        vmax=height_avg + std_multiplier * height_std)
    axes[0, 2].set_title("height [m]")
    axes[0, 2].yaxis.set_tick_params(labelleft=False)
    fig.colorbar(ax3, ax=axes[0, 2], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
else:
    axes[0, 2].set_title("height [m]\n[not available]")
    axes[0, 2].axis('off')

# Plot incidenceAngle
if hasattr(product_secondary, "geometry_incidenceAngle"):
    incidenceAngle = product_secondary1.geometry_incidenceAngle
    incidenceAngle_avg = np.mean(incidenceAngle)
    incidenceAngle_std = np.std(incidenceAngle)
    ax4 = axes[1, 0].imshow(incidenceAngle, cmap='rainbow', aspect=aspect_ratio,
        vmin=incidenceAngle_avg - std_multiplier * incidenceAngle_std,
        vmax=incidenceAngle_avg + std_multiplier * incidenceAngle_std)
    axes[1, 0].set_title("incidenceAngle [deg]")
    fig.colorbar(ax4, ax=axes[1, 0], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
else:
    axes[1, 0].set_title("incidenceAngle [deg]\n[not available]")
    axes[1, 0].axis('off')

# Plot elevationAngle
if hasattr(product_secondary, "geometry_elevationAngle"):
    elevationAngle = product_secondary1.geometry_elevationAngle
    elevationAngle_avg = np.mean(elevationAngle)
    elevationAngle_std = np.std(elevationAngle)
    ax5 = axes[1, 1].imshow(elevationAngle, cmap='rainbow', aspect=aspect_ratio,
        vmin=elevationAngle_avg - std_multiplier * elevationAngle_std,
        vmax=elevationAngle_avg + std_multiplier * elevationAngle_std)
    axes[1, 1].set_title("elevationAngle [deg]")
    axes[1, 1].yaxis.set_tick_params(labelleft=False)
    fig.colorbar(ax5, ax=axes[1, 1], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
else:
    axes[1, 1].set_title("elevationAngle [deg]\n[not available]")
    axes[1, 1].axis('off')

# Plot terrainSlope
if hasattr(product_secondary, "geometry_terrainSlope"):
    terrainSlope = product_secondary1.geometry_terrainSlope
    terrainSlope_avg = np.mean(terrainSlope)
    terrainSlope_std = np.std(terrainSlope)
    ax6 = axes[1, 2].imshow(terrainSlope, cmap='rainbow', aspect=aspect_ratio,
        vmin=terrainSlope_avg - std_multiplier * terrainSlope_std,
        vmax=terrainSlope_avg + std_multiplier * terrainSlope_std)
    axes[1, 2].set_title("terrainSlope [°]")
    axes[1, 2].yaxis.set_tick_params(labelleft=False)
    fig.colorbar(ax6, ax=axes[1, 2], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
else:
    axes[1, 2].set_title("terrainSlope [°]\n[not available]")
    axes[1, 2].axis('off')

fig.tight_layout()
plt.show()

#### Section 2.4: Print LUT coregistration group

This includes (Requires an upsampling before its application on the original COG temporal grid):
1. azimuthCoregistrationShifts
2. rangeCoregistrationShifts
3. coregistrationShiftsQuality
4. waveNumbers  #Values are expressed in [rad/m]
5. flatteningPhasesScreen #Values are expressed in [rad].

In [ ]:
#plot azimuthCoregistrationShifts, rangeCoregistrationShifts, coregistrationShiftsQuality

std_multiplier = 3
aspect_ratio = 'auto' #0.5
colorbar_fraction = 0.05
colorbar_pad = 0.04

plt.close('all')
fig, axes = plt.subplots(2, 3, figsize=(16, 14))

# Azimuth Coregistration Shifts
if hasattr(product_secondary, "coregistration_azimuthCoregistrationShifts"):
    azimuthCoregistrationShifts  = product_secondary1.coregistration_azimuthCoregistrationShifts
    azimuthCoregistrationShifts_avg = np.mean(np.ma.masked_equal(azimuthCoregistrationShifts,noDataValue))
    azimuthCoregistrationShifts_std = np.std(np.ma.masked_equal(azimuthCoregistrationShifts, noDataValue))

    ax1 = axes[0, 0].imshow(azimuthCoregistrationShifts, cmap='rainbow', aspect=aspect_ratio,
        vmin=azimuthCoregistrationShifts_avg - std_multiplier * azimuthCoregistrationShifts_std,
        vmax=azimuthCoregistrationShifts_avg + std_multiplier * azimuthCoregistrationShifts_std)
    axes[0, 0].set_title("Azimuth Coreg Shifts [m]")
    fig.colorbar(ax1, ax=axes[0, 0], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
else:
    axes[0, 0].set_title("Azimuth Coreg Shifts\n[not available]")
    axes[0, 0].axis('off')

# Range Coregistration Shifts
if hasattr(product_secondary, "coregistration_rangeCoregistrationShifts"):
    rangeCoregistrationShifts    = product_secondary1.coregistration_rangeCoregistrationShifts
    rangeCoregistrationShifts_avg = np.mean(np.ma.masked_equal(rangeCoregistrationShifts, noDataValue))
    rangeCoregistrationShifts_std = np.std(np.ma.masked_equal(rangeCoregistrationShifts,noDataValue))

    ax2 = axes[0, 1].imshow(rangeCoregistrationShifts, cmap='rainbow', aspect=aspect_ratio,
        vmin=rangeCoregistrationShifts_avg - std_multiplier * rangeCoregistrationShifts_std,
        vmax=rangeCoregistrationShifts_avg + std_multiplier * rangeCoregistrationShifts_std)
    axes[0, 1].set_title("Range Coreg Shifts [m]")
    axes[0, 1].yaxis.set_tick_params(labelleft=False)
    fig.colorbar(ax2, ax=axes[0, 1], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
else:
    axes[0, 1].set_title("Range Coreg Shifts\n[not available]")
    axes[0, 1].axis('off')

# Coregistration Shifts Quality
if hasattr(product_secondary, "coregistration_coregistrationShiftsQuality"):
    coregistrationShiftsQuality  = product_secondary1.coregistration_coregistrationShiftsQuality
    coregistrationShiftsQuality_avg = np.mean(np.ma.masked_equal(coregistrationShiftsQuality, noDataValue))
    coregistrationShiftsQuality_std = np.std(np.ma.masked_equal(coregistrationShiftsQuality, noDataValue))
    ax3 = axes[0, 2].imshow(coregistrationShiftsQuality, cmap='rainbow', aspect=aspect_ratio,
        vmin=coregistrationShiftsQuality_avg - std_multiplier * coregistrationShiftsQuality_std,
        vmax=coregistrationShiftsQuality_avg + std_multiplier * coregistrationShiftsQuality_std)
    axes[0, 2].set_title("Coreg Shifts Quality")
    axes[0, 2].yaxis.set_tick_params(labelleft=False)
    fig.colorbar(ax3, ax=axes[0, 2], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
else:
    axes[0, 2].set_title("Coreg Shifts Quality\n[not available]")
    axes[0, 2].axis('off')

# Wave Numbers
if hasattr(product_secondary, "coregistration_waveNumbers"):
    waveNumbers                  = product_secondary1.coregistration_waveNumbers 
    waveNumbers_avg = np.mean(np.ma.masked_equal(waveNumbers,noDataValue))
    waveNumbers_std = np.std(np.ma.masked_equal(waveNumbers, noDataValue))
    ax4 = axes[1, 0].imshow(waveNumbers, cmap='rainbow', aspect=aspect_ratio,
        vmin=waveNumbers_avg - std_multiplier * waveNumbers_std,
        vmax=waveNumbers_avg + std_multiplier * waveNumbers_std)
    axes[1, 0].set_title("Vertical Wave Numbers [rad/m]")
    fig.colorbar(ax4, ax=axes[1, 0], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
else:
    axes[1, 0].set_title("Wave Numbers [rad/m]\n[not available]")
    axes[1, 0].axis('off')

# Flattening Phase Screen
if hasattr(product_secondary, "coregistration_flatteningPhaseScreen"):
    flatteningPhaseScreen       = product_secondary1.coregistration_flatteningPhaseScreen
    flatteningPhaseScreen_avg = np.mean(np.ma.masked_equal(flatteningPhaseScreen,noDataValue))
    flatteningPhaseScreen_std = np.std(np.ma.masked_equal(flatteningPhaseScreen, noDataValue))
    ax5 = axes[1, 1].imshow(flatteningPhaseScreen, cmap='rainbow', aspect=aspect_ratio,
        vmin=flatteningPhaseScreen_avg - std_multiplier * flatteningPhaseScreen_std,
        vmax=flatteningPhaseScreen_avg + std_multiplier * flatteningPhaseScreen_std)
    axes[1, 1].set_title("Flattening Phase Screen [rad]")
    axes[1, 1].yaxis.set_tick_params(labelleft=False)
    fig.colorbar(ax5, ax=axes[1, 1], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
else:
    axes[1, 1].set_title("Flattening Phase Screen\n[not available]")
    axes[1, 1].axis('off')

# Last panel vuoto
axes[1, 2].axis('off')

fig.tight_layout()
plt.show()

#### Section 2.4: Print LUT skpPhaseCalibration group 

This includes (Requires an upsampling before its application on the original COG temporal grid):
1. groundPhasesScreen 
2. groundPhasesScreenQuality

In [ ]:

std_multiplier = 3
aspect_ratio = 'auto'
colorbar_fraction = 0.05
colorbar_pad = 0.04

plt.close('all')

fig, axes = plt.subplots(1,2, figsize=(7,6)) 
# skpCalibrationPhaseScreen
if hasattr(product_secondary, "skpPhaseCalibration_skpCalibrationPhaseScreen"):
    skpCalibrationPhaseScreen = product_secondary1.skpPhaseCalibration_skpCalibrationPhaseScreen
    skpCalibrationPhaseScreen_avg = np.mean(np.ma.masked_equal(skpCalibrationPhaseScreen, noDataValue))
    skpCalibrationPhaseScreen_std = np.std(np.ma.masked_equal(skpCalibrationPhaseScreen, noDataValue))
    
    ax1 = axes[0].imshow(skpCalibrationPhaseScreen, cmap='rainbow', aspect=aspect_ratio,
        vmin=skpCalibrationPhaseScreen_avg - std_multiplier * skpCalibrationPhaseScreen_std,
        vmax=skpCalibrationPhaseScreen_avg + std_multiplier * skpCalibrationPhaseScreen_std)
    axes[0].set_title("skpCalibrationPhasesScreen [deg]")
    fig.colorbar(ax1, ax=axes[0], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
else:
    axes[0].set_title("SKP Phase Calibration\n[not available]")
    axes[0].axis('off')

# skpCalibrationPhaseScreenQuality
if hasattr(product_secondary, "skpPhaseCalibration_skpCalibrationPhaseScreenQuality"):
    skpCalibrationPhaseScreenQuality = product_secondary1.skpPhaseCalibration_skpCalibrationPhaseScreenQuality
    skpCalibrationPhaseScreenQuality_avg = np.mean(np.ma.masked_equal(skpCalibrationPhaseScreenQuality, noDataValue))
    skpCalibrationPhaseScreenQuality_std = np.std(np.ma.masked_equal(skpCalibrationPhaseScreenQuality, noDataValue))
    
    ax2 = axes[1].imshow(skpCalibrationPhaseScreenQuality, cmap='rainbow', aspect=aspect_ratio,
        vmin=skpCalibrationPhaseScreenQuality_avg - std_multiplier * skpCalibrationPhaseScreenQuality_std,
        vmax=skpCalibrationPhaseScreenQuality_avg + std_multiplier * skpCalibrationPhaseScreenQuality_std)
    axes[1].set_title("skpCalibrationPhasesScreenQuality")
    axes[1].yaxis.set_tick_params(labelleft=False)
    fig.colorbar(ax2, ax=axes[1], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
else:
    axes[1].set_title("SKP Calibration Flag\n[not available]")
    axes[1].axis('off')


fig.tight_layout()
plt.show()


#### Section 2.5: Print LUT baselineAndIonosphereCorrection group 

This includes (Requires an upsampling before its application on the original COG temporal grid):
1. baselineErrorPhaseScreen
2. residualIonospherePhaseScreen
3. residualIonospherePhaseScreenQuality

In [ ]:
#plot  baselineAndIonosphereCorrection 

std_multiplier = 3
aspect_ratio = 'auto'
colorbar_fraction = 0.05
colorbar_pad = 0.04

plt.close('all')
fig, axes = plt.subplots(1,3, figsize=(8,6))


# 1. Baseline Error Phase Screen
if hasattr(product_secondary, "baselineAndIonosphereCorrection_baselineErrorPhaseScreen"):
    baselineErrorPhaseScreen = product_secondary1.baselineAndIonosphereCorrection_baselineErrorPhaseScreen
    baselineErrorPhaseScreen_avg = np.mean(np.ma.masked_equal(baselineErrorPhaseScreen, noDataValue))
    baselineErrorPhaseScreen_std = np.std(np.ma.masked_equal(baselineErrorPhaseScreen, noDataValue))

    ax1 = axes[0].imshow(baselineErrorPhaseScreen, cmap='rainbow', aspect=aspect_ratio,
        vmin=baselineErrorPhaseScreen_avg - std_multiplier * baselineErrorPhaseScreen_std,
        vmax=baselineErrorPhaseScreen_avg + std_multiplier * baselineErrorPhaseScreen_std)
    axes[0].set_title("Baseline Error\nPhase Screen [rad]")
    fig.colorbar(ax1, ax=axes[0], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
else:
    axes[0].set_title("Baseline Error\nPhase Screen\n[not available]")
    axes[0].axis('off')

# 2. Residual Ionosphere Phase Screen
if hasattr(product_secondary, "baselineAndIonosphereCorrection_residualIonospherePhaseScreen"):
    residualIonospherePhaseScreen = product_secondary1.baselineAndIonosphereCorrection_residualIonospherePhaseScreen
    residualIonospherePhaseScreen_avg = np.mean(np.ma.masked_equal(residualIonospherePhaseScreen, noDataValue))
    residualIonospherePhaseScreen_std = np.std(np.ma.masked_equal(residualIonospherePhaseScreen, noDataValue))

    ax2 = axes[1].imshow(residualIonospherePhaseScreen, cmap='rainbow', aspect=aspect_ratio,
        vmin=residualIonospherePhaseScreen_avg - std_multiplier * residualIonospherePhaseScreen_std,
        vmax=residualIonospherePhaseScreen_avg + std_multiplier * residualIonospherePhaseScreen_std)
    axes[1].set_title("Residual Ionosphere\nPhase Screen [rad]")
    axes[1].yaxis.set_tick_params(labelleft=False)
    fig.colorbar(ax2, ax=axes[1], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
else:
    axes[1].set_title("Residual Ionosphere\nPhase Screen \n[not available]")
    axes[1].axis('off')

# 3. Residual Ionosphere Phase Screen Quality
if hasattr(product_secondary, "baselineAndIonosphereCorrection_residualIonospherePhaseScreenQuality"):
    residualIonospherePhaseScreenQuality = product_secondary1.baselineAndIonosphereCorrection_residualIonospherePhaseScreenQuality
    residualIonospherePhaseScreenQuality_avg = np.mean(np.ma.masked_equal(residualIonospherePhaseScreenQuality, noDataValue))
    residualIonospherePhaseScreenQuality_std = np.std(np.ma.masked_equal(residualIonospherePhaseScreenQuality, noDataValue))

    ax3 = axes[2].imshow(residualIonospherePhaseScreenQuality, cmap='rainbow', aspect=aspect_ratio,
        vmin=residualIonospherePhaseScreenQuality_avg - std_multiplier * residualIonospherePhaseScreenQuality_std,
        vmax=residualIonospherePhaseScreenQuality_avg + std_multiplier * residualIonospherePhaseScreenQuality_std)
    axes[2].set_title("Residual Ionosphere\nPhase Screen Quality")
    axes[2].yaxis.set_tick_params(labelleft=False)
    fig.colorbar(ax3, ax=axes[2], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
else:
    axes[2].set_title("Residual Ionosphere Phase\nScreen Quality\n[not available]")
    axes[2].axis('off')

fig.tight_layout()
plt.show()

# 🍃 Section 3:  upsampling of luts into the slc grid


In [ ]:
import xml.etree.ElementTree as ET  #needed to fetch  the data from annotations
import rasterio
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
import warnings
import scipy as sp
from pathlib import Path
import BiomassProduct
warnings.filterwarnings('ignore', category=DeprecationWarning)

#### Section 3.1: paths (both primary and secondary)

In [ ]:
#folder_TDS = Path("..") / "L1c"
folder_TDS = Path(r"WD_F158_1C")
print (folder_TDS)
loader = BiomassProduct.STAProductGroupLoader(folder_TDS)
loader.print_summary()
product_primary = loader.get_instance_by_name("product_primary")
product_secondary = loader.get_instance_by_name("product_secondary1")

# Access and print the noDataValue from the primary product's annotation
noDataValue=product_primary.noDataValue
print("NoData Value (primary product):", noDataValue)

path_main_ann_coregistered = product_secondary.annotation_coregistered_xml_file
path_main_ann_primary = product_primary.annotation_coregistered_xml_file

print (path_main_ann_coregistered)
print (path_main_ann_primary)


#### Section 3.2: get the relevant data (both primary and secondary)


##### Slant range
- firstSampleSlantRangeTime
- rangeTimeInterval
- numberOfSamples
- lut axes

##### Azimuth 
- azimuthTimeInterval
- firstLineAzimuthTime
- numberOfLines
- lut axes

In [ ]:
# Coordinate asse azimutale e range del prodotto co-registrato
relativeAzimuthTime_co   = product_secondary.relativeAzimuthTime
slantRangeTime_co        = product_secondary.slantRangeTime

relativeAzimuthTime_pri  = product_primary.relativeAzimuthTime
slantRangeTime_pri       = product_primary.slantRangeTime

lut_az_axes_co = (relativeAzimuthTime_co - relativeAzimuthTime_co[0]).astype(np.float64)
lut_az_axes_pri = (relativeAzimuthTime_pri - relativeAzimuthTime_pri[0]).astype(np.float64)

lut_range_axis_co = slantRangeTime_co - slantRangeTime_co[0]
lut_range_axis_pri = slantRangeTime_pri - slantRangeTime_pri[0]

print('lut_az_axes_co =     '+str(relativeAzimuthTime_co[0]))

#### Section 3.2: --> axes from coregistered data == axes from primary data

In [ ]:
assert np.array_equal(lut_az_axes_co, lut_az_axes_pri)
assert np.array_equal(lut_range_axis_co, lut_range_axis_pri)

#### Section 3.2: --> parse the main annotation

In [ ]:
main_ann_coregistered = ET.parse(path_main_ann_coregistered)
main_ann_coregistered_root = main_ann_coregistered.getroot()

main_ann_primary = ET.parse(path_main_ann_primary)
main_ann_primary_root = main_ann_primary.getroot()

#### Section 3.2: --> get the data from the main annotations (primary and coregistered slc (L1C))

In [ ]:
sarImage_co = main_ann_coregistered_root.findall('sarImage')
#range coreg
firstSampleSlantRangeTime_co = np.float64(sarImage_co[0].findall("firstSampleSlantRangeTime")[0].text)
rangeTimeInterval_co = np.float64(sarImage_co[0].findall("rangeTimeInterval")[0].text)
numberOfSamples_co = np.int64(sarImage_co[0].findall("numberOfSamples")[0].text)

#az coreg
azimuthTimeInterval_co = np.float64(sarImage_co[0].findall("azimuthTimeInterval")[0].text)
firstLineAzimuthTime_co = np.datetime64(sarImage_co[0].findall("firstLineAzimuthTime")[0].text)
numberOfLines_co = np.int64(sarImage_co[0].findall("numberOfLines")[0].text)


sarImage_pri = main_ann_primary_root.findall('sarImage')
#range coreg
firstSampleSlantRangeTime_pri = np.float64(sarImage_pri[0].findall("firstSampleSlantRangeTime")[0].text)
rangeTimeInterval_pri = np.float64(sarImage_pri[0].findall("rangeTimeInterval")[0].text)
numberOfSamples_pri = np.int64(sarImage_pri[0].findall("numberOfSamples")[0].text)

#az coreg
azimuthTimeInterval_pri = np.float64(sarImage_pri[0].findall("azimuthTimeInterval")[0].text)
firstLineAzimuthTime_pri = np.datetime64(sarImage_pri[0].findall("firstLineAzimuthTime")[0].text)
numberOfLines_pri = np.int64(sarImage_pri[0].findall("numberOfLines")[0].text)


#### Section 3.2: --> coregistered extracted data == primary extracted data

In [ ]:
assert firstSampleSlantRangeTime_co == firstSampleSlantRangeTime_pri
assert rangeTimeInterval_co == rangeTimeInterval_pri
assert numberOfSamples_co == numberOfSamples_pri
assert azimuthTimeInterval_co == azimuthTimeInterval_pri
assert numberOfLines_co == numberOfLines_pri

#### Section 3.3: calculate slc axes

In [ ]:
######################## as per STA_P
#coreg az axis
axis = 0
roi_co =  [0, 0,numberOfLines_co, numberOfSamples_co]
time_step_co = azimuthTimeInterval_co
time_start_co = 0
az_slc_axis_co_stac = np.arange(roi_co[axis + 2], dtype=np.float64) * time_step_co

#primary az axis
axis = 0
roi_pri =  [0, 0,numberOfLines_pri, numberOfSamples_pri]
time_step_pri = azimuthTimeInterval_pri
time_start_pri = 0
az_slc_axis_pri_stac = np.arange(roi_pri[axis + 2], dtype=np.float64) * time_step_pri



#coreg rg axis
axis = 1
roi_co =  [0, 0,numberOfLines_co, numberOfSamples_co]
time_step_co = rangeTimeInterval_co
time_start_co = firstSampleSlantRangeTime_co
range_slc_axis_co_stac = np.arange(roi_co[axis + 2], dtype=np.float64) * time_step_co

#pri rg axis
axis = 1
roi_pri =  [0, 0,numberOfLines_pri, numberOfSamples_pri]
time_step_pri = rangeTimeInterval_pri
time_start_pri = firstSampleSlantRangeTime_pri
range_slc_axis_pri_stac = np.arange(roi_pri[axis + 2], dtype=np.float64) * time_step_pri

# Print result clearly
print("\n[INFO] SLC Primary Range Axis:")
print(f"  - Number of samples : {numberOfSamples_pri}")
print(f"  - Range spacing     : {rangeTimeInterval_pri} seconds")
print(f"  - Max range axis    : {np.max(range_slc_axis_pri_stac):.6f} seconds")

# Print result clearly
print("\n[INFO] SLC Secondary Range Axis:")
print(f"  - Number of samples : {numberOfSamples_co}")
print(f"  - Range spacing     : {rangeTimeInterval_co} seconds")
print(f"  - Max range axis    : {np.max(range_slc_axis_co_stac):.6f} seconds")

#### Section 3.3: primary axes == secondary axes

In [ ]:
assert np.array_equal(az_slc_axis_co_stac, az_slc_axis_pri_stac)
assert np.array_equal(range_slc_axis_co_stac, range_slc_axis_pri_stac)

#### Section 3.4: get some data to interpolate (see section 2 for details)

In [ ]:
flatteningPhaseScreen_co = product_secondary.coregistration_flatteningPhaseScreen
skpCalibrationPhaseScreen_co = product_secondary.skpPhaseCalibration_skpCalibrationPhaseScreen
height_co = product_secondary.geometry_height
sigmaNought_co = product_secondary.radiometry_sigmaNought
gammaNought_co = product_secondary.radiometry_gammaNought

#### Section 3.5: preparation of the inputs for the interpolation

from stack processor bps-stack_cal_processor\bps\stack_cal_processor\core\utils.py function def _interpolate_2d

see also https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.RectBivariateSpline.html for details

In [ ]:
axes_in = (lut_az_axes_pri, lut_range_axis_pri) # as showed before you can use also the *co axes since they are equal as expteced 
axes_out = (az_slc_axis_pri_stac, range_slc_axis_pri_stac) # as showed before you can use also the *co axes since they are equal as expteced 
degree_x = 1
degree_y = 1
smoother = 0.0
bbox=[
            min(np.min(axes_in[0]), np.min(axes_out[0])),
            max(np.max(axes_in[0]), np.max(axes_out[0])),
            max(np.min(axes_in[1]), np.min(axes_out[1])),
            max(np.max(axes_in[1]), np.max(axes_out[1])),
        ]

#### Section 3.6: let's define the interpolator object - example with flatteningPhasesScreen

In [ ]:
flatteningPhaseScreen_co_interpolator = sp.interpolate.RectBivariateSpline(    
            axes_in[0],
            axes_in[1],
            flatteningPhaseScreen_co,
            bbox= bbox,
        kx=degree_x,
            ky=degree_y,
            s=smoother
         )

flatteningPhaseScreen_co_upsampled = flatteningPhaseScreen_co_interpolator(axes_out[0], axes_out[1])

#### Section 3.6: let's define the interpolator object - example with skpCalibrationPhaseScreen # --- upsample SKP as exp(j·skp) ---

In [ ]:
def upsample_phase_via_complex(phase_in: np.ndarray,
                               axes_in: tuple[np.ndarray, np.ndarray],
                               axes_out: tuple[np.ndarray, np.ndarray],
                               bbox: list[float],
                               kx: int = 1, ky: int = 1, s: float = 0.0,
                               nodata: float = -9999.0) -> np.ndarray:
    """
     Upsampling of a phase (in radians) using interpolation on cosine and sine, 
     to avoid artifacts caused by phase wrapping.
    """

    # Convert to real and imaginary parts of e^(j*phase)
    real_part = np.cos(phase_in)
    imag_part = np.sin(phase_in)

    interp_re = sp.interpolate.RectBivariateSpline(
        axes_in[0], axes_in[1], real_part,
        bbox=bbox, kx=kx, ky=ky, s=s
    )
    interp_im = sp.interpolate.RectBivariateSpline(
        axes_in[0], axes_in[1], imag_part,
        bbox=bbox, kx=kx, ky=ky, s=s
    )

    re_up = interp_re(axes_out[0], axes_out[1])
    im_up = interp_im(axes_out[0], axes_out[1])

    # Reconstruct the complex number e^(jφ) and extract its angle
    comp = re_up + 1j * im_up
    phase_up = np.angle(comp)

    return phase_up

# --- upsample SKP as exp(j·skp) ---
skpCalibrationPhaseScreen_co_upsampled = upsample_phase_via_complex(
        skpCalibrationPhaseScreen_co,
        axes_in=(lut_az_axes_pri, lut_range_axis_pri),
        axes_out=(az_slc_axis_pri_stac, range_slc_axis_pri_stac),
        bbox=bbox, kx=degree_x, ky=degree_y, s=smoother, nodata=-9999.0
        )


#### Section 3.6: let's define the interpolator object - example with height

In [ ]:
height_co_interpolator= sp.interpolate.RectBivariateSpline(    
        axes_in[0],
        axes_in[1],
        height_co,
        bbox= bbox,
    kx=degree_x,
        ky=degree_y,
        s=smoother
     )

height_co_upsampled = height_co_interpolator(axes_out[0], axes_out[1])


In [ ]:
sigmaNought_co_interpolator = sp.interpolate.RectBivariateSpline(    
        axes_in[0],
        axes_in[1],
        sigmaNought_co,
        bbox= bbox,
    kx=degree_x,
        ky=degree_y,
        s=smoother
     )
sigmaNought_co_upsampled = sigmaNought_co_interpolator(axes_out[0], axes_out[1])


In [ ]:
gammaNought_co_interpolator= sp.interpolate.RectBivariateSpline(    
        axes_in[0],
        axes_in[1],
        gammaNought_co,
        bbox= bbox,
    kx=degree_x,
        ky=degree_y,
        s=smoother
     )
gammaNought_co_upsampled = gammaNought_co_interpolator(axes_out[0], axes_out[1])


#### Section 3.8 Generation of Gamma Nought and Sigma Nought

In [ ]:
path_amp= product_secondary.measurement_abs_file
nan_value = -9999
with rasterio.open(path_amp) as amp_raster: #get the amp channels
    amp_hh = amp_raster.read(1) #nupy.ndarray object
    amp_hh = np.ma.masked_equal(amp_hh, nan_value)

According to the L1_PFD
- gammaNought_hh (linear) = [amp_hh^2 ] * gammaNought_conversion_factor
- sigmaNought_hh (linear) = [amp_hh^2 ] * sigmaNought_conversion_factor

Below they are proposed in db:e.g.
10*log10(gammaNought_hh) = 20*log10(amp) + 10*log10(conversion_factor)

In [ ]:
gammaNought_hh_db = 20 * np.log10(amp_hh) + 10*np.log10(gammaNought_co_upsampled)
sigmaNought_hh_db = 20 * np.log10(amp_hh) +  10*np.log10(sigmaNought_co_upsampled)

#### Section 3.9: Plot the results and compare them with the original COG grid 
Beta Noought and upsampled flatteningPhasesScreen and height

In [ ]:
betaNought_hh_db = 20*np.log10(amp_hh)
betaNought_hh_db_avg = np.mean(betaNought_hh_db[2000:25000, :])
betaNought_hh_db_std = np.std(betaNought_hh_db[2000:25000, :])

upsampled_array_flatteningPhasesScreen_avg = np.mean(np.ma.masked_equal(flatteningPhaseScreen_co_upsampled,noDataValue))
upsampled_array_flatteningPhasesScreen_std = np.std(np.ma.masked_equal(flatteningPhaseScreen_co_upsampled, noDataValue))

skpCalibrationPhaseScreen_co_upsampled_avg = np.mean(np.ma.masked_equal(skpCalibrationPhaseScreen_co_upsampled,noDataValue))
skpCalibrationPhaseScreen_co_upsampled_std = np.std(np.ma.masked_equal(skpCalibrationPhaseScreen_co_upsampled, noDataValue))

upsampled_array_height_avg = np.mean(height_co_upsampled)
upsampled_array_height_std = np.std(height_co_upsampled)

std_multiplier = 3
aspect_ratio = 'auto' #0.1
colorbar_fraction = 0.05
colorbar_pad = 0.04

plt.close('all')
fig, axes = plt.subplots(1,4, figsize=(16,6)) 

ax1 = axes[0].imshow( 
    betaNought_hh_db, 
    cmap='rainbow',
    aspect=aspect_ratio,
    vmin =  betaNought_hh_db_avg - std_multiplier* betaNought_hh_db_std,
    vmax =  betaNought_hh_db_avg + std_multiplier* betaNought_hh_db_std
)
axes[0].set_title('Beta Nought HH db')


ax2 = axes[1].imshow( 
    flatteningPhaseScreen_co_upsampled, 
    cmap='rainbow',
    aspect=aspect_ratio,
    vmin =  upsampled_array_flatteningPhasesScreen_avg - std_multiplier* upsampled_array_flatteningPhasesScreen_std,
    vmax =  upsampled_array_flatteningPhasesScreen_avg + std_multiplier* upsampled_array_flatteningPhasesScreen_std,
    interpolation="none"
)
axes[1].set_title('flatteningPhasesScreen [rad]')
axes[1].yaxis.set_tick_params(labelleft=False)


ax3 = axes[2].imshow( 
    skpCalibrationPhaseScreen_co_upsampled, 
    cmap='rainbow',
    aspect=aspect_ratio,
    vmin =  skpCalibrationPhaseScreen_co_upsampled_avg - std_multiplier* skpCalibrationPhaseScreen_co_upsampled_std,
    vmax =  skpCalibrationPhaseScreen_co_upsampled_avg + std_multiplier* skpCalibrationPhaseScreen_co_upsampled_std,
    interpolation="none"
)
axes[2].set_title('skpCalibrationPhaseScreen [rad]')
axes[2].yaxis.set_tick_params(labelleft=False)


ax4 = axes[3].imshow( 
    height_co_upsampled, 
    cmap='rainbow',
    aspect=aspect_ratio,
    vmin =  upsampled_array_height_avg - std_multiplier* upsampled_array_height_std,
    vmax =  upsampled_array_height_avg + std_multiplier* upsampled_array_height_std
)
axes[3].set_title('height [m]')
axes[3].yaxis.set_tick_params(labelleft=False)


fig.colorbar(ax3, ax=axes[2], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.colorbar(ax1, ax=axes[0], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.colorbar(ax2, ax=axes[1], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)           
fig.colorbar(ax4, ax=axes[3], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad) 

fig.tight_layout()
plt.show()

#### Section 3.9: Plot the results and compare them with the original COG grid 
Beta Nought and upsampled Gamma Nought and Sigma Nought

In [ ]:
betaNought_hh_db = 20*np.log10(amp_hh)
betaNought_hh_db_avg = np.mean(betaNought_hh_db[2000:25000, :])
betaNought_hh_db_std = np.std(betaNought_hh_db[2000:25000, :])

gammaNought_hh_db_avg = np.mean(gammaNought_hh_db[2000:25000, :])
gammaNought_hh_db_std = np.std(gammaNought_hh_db[2000:25000, :])

sigmaNought_hh_db_avg = np.mean(sigmaNought_hh_db[2000:25000, :])
sigmaNought_hh_db_std = np.std(sigmaNought_hh_db[2000:25000, :])

std_multiplier = 3
aspect_ratio = 'auto'#0.1
colorbar_fraction = 0.05
colorbar_pad = 0.04

plt.close('all')
fig, axes = plt.subplots(1,3, figsize=(8,6)) 

ax1 = axes[0].imshow( 
    betaNought_hh_db, 
    cmap='rainbow',
    aspect=aspect_ratio,
    vmin =  betaNought_hh_db_avg - std_multiplier* betaNought_hh_db_std,
    vmax =  betaNought_hh_db_avg + std_multiplier* betaNought_hh_db_std
)
axes[0].set_title('Beta Nought HH db')


ax2 = axes[1].imshow( 
    gammaNought_hh_db, 
    cmap='rainbow',
    aspect=aspect_ratio,
    vmin =  gammaNought_hh_db_avg - std_multiplier* gammaNought_hh_db_std,
    vmax =  gammaNought_hh_db_avg + std_multiplier* gammaNought_hh_db_std
)
axes[1].set_title('Gamma Nought HH db')
axes[1].yaxis.set_tick_params(labelleft=False)


ax3 = axes[2].imshow( 
    sigmaNought_hh_db, 
    cmap='rainbow',
    aspect=aspect_ratio,
    vmin =  sigmaNought_hh_db_avg - std_multiplier* sigmaNought_hh_db_std,
    vmax =  sigmaNought_hh_db_avg + std_multiplier* sigmaNought_hh_db_std
)
axes[2].set_title('Sigma Nought HH db')
axes[2].yaxis.set_tick_params(labelleft=False)


fig.colorbar(ax3, ax=axes[2], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.colorbar(ax1, ax=axes[0], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.colorbar(ax2, ax=axes[1], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)                

fig.tight_layout()
plt.show()

# 🌲Section 4: Basic Interferometry

#### 4.1 Get the data and reconstruct the complex information 

In [ ]:
import xml.etree.ElementTree as ET  #needed to fetch  the data from annotations
import rasterio
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
import warnings
import scipy as sp
from scipy.stats import gaussian_kde
from support_functions_l1c import single_baseline_single_pol_coh
import BiomassProduct
from pathlib import Path

warnings.filterwarnings('ignore', category=DeprecationWarning)

## Load BIOMASS STA Products

We define the paths to the primary and secondary BIOMASS STA products and instantiate `BiomassProductSTA` objects for each. This automatically loads LUT paths, annotation XML, and measurement files (amplitude and phase).

In [ ]:
folder_TDS = Path(r"WD_F158_1C") 

loader = BiomassProduct.STAProductGroupLoader(folder_TDS)
loader.print_summary()
product_primary = loader.get_instance_by_name("product_primary")
product_secondary = loader.get_instance_by_name("product_secondary1")

path_main_ann_coregistered = product_secondary.annotation_coregistered_xml_file
path_main_ann_primary = product_primary.annotation_coregistered_xml_file

print (path_main_ann_coregistered)
print (path_main_ann_primary)

# Access and print the noDataValue from the primary product's annotation
noDataValue=product_primary.noDataValue
print("NoData Value (primary product):", noDataValue)


data_primary_abs = product_primary.measurement_abs_file             
data_primary_phase = product_primary.measurement_phase_file 
path_lut_primary = product_primary.annotation_coregistered_lut_file         
path_main_ann_primary = product_primary.annotation_coregistered_xml_file    

data_secondary_abs = product_secondary.measurement_abs_file 
data_secondary_phase = product_secondary.measurement_phase_file  
path_lut_coregistered = product_secondary.annotation_coregistered_lut_file   
path_main_ann_coregistered = product_secondary.annotation_coregistered_xml_file


print(data_primary_abs)
print(path_lut_primary)
print(path_main_ann_primary)
print(data_secondary_phase)
print(path_lut_coregistered)
print(path_main_ann_coregistered)

def _get_bool(root, xpath):
    node = root.find(xpath)
    return (node is not None) and (node.text or "").strip().lower() == "true"

# Parse XML and extract primary/secondary SCS names
tree = ET.parse(path_main_ann_primary)
root = tree.getroot()

skpPhaseCalibrationFlag = _get_bool(root, ".//skpPhaseCalibrationFlag")
skpPhaseCorrectionFlag = _get_bool(root, ".//skpPhaseCorrectionFlag")        
skpPhaseCorrectionFlatteningOnlyFlag = _get_bool( root, ".//skpPhaseCorrectionFlatteningOnlyFlag")
        
print(f"skpPhaseCalibrationFlag: {skpPhaseCalibrationFlag}")
print(f"skpPhaseCorrectionFlag: {skpPhaseCorrectionFlag}")
print(f"skpPhaseCorrectionFlatteningOnlyFlag: {skpPhaseCorrectionFlatteningOnlyFlag}")

if skpPhaseCorrectionFlag:
    if skpPhaseCorrectionFlatteningOnlyFlag:
        print( "Flattening Phase Screen")
        flatten = 'geometry'
    else:
        print("Ground Phase Screen")
        flatten = 'skp'
else:
    print("No Phase Screen")
    flatten = "None"
        
print(f"[INFO] phaseCorrection mode = {flatten}")


In [ ]:

no_data_abs_phase =noDataValue
nodata = (no_data_abs_phase * np.exp(1j * no_data_abs_phase)).astype(np.complex64)

def read_complex_channels(abs_path: str, phase_path: str):
    """
    Reads the complex SAR channels (HH, XX, VV) by reconstructing them from amplitude and phase data.

    Parameters:
        abs_path (str): Path to the amplitude GeoTIFF file.
        phase_path (str): Path to the phase GeoTIFF file.

    Returns:
        Tuple of complex numpy arrays: (HH, XX, VV), each with shape (H, W)
    """
    abs_path = Path(abs_path)
    phase_path = Path(phase_path)
    
    # Open amplitude and phase raster files
    with rasterio.open(abs_path) as abs_src, rasterio.open(phase_path) as phase_src:
        abs_data = abs_src.read()    # shape: (3, H, W)
        phase_data = phase_src.read()

    # Reconstruct complex values: amplitude * exp(j * phase)
    complex_data = abs_data * np.exp(1j * phase_data)
    # Extract each polarization channel
    hh = complex_data[0]
    xx = complex_data[1]  
    vv = complex_data[2]

    return hh, xx, vv

hh_slc_pri, xx_slc_pri, vv_slc_pri = read_complex_channels(data_primary_abs, data_primary_phase)
hh_slc_sec, xx_slc_sec, vv_slc_sec = read_complex_channels(data_secondary_abs, data_secondary_phase)


In [ ]:
print(noDataValue*np.exp(1j * noDataValue))

Generate athe complex data

#### 4.2 Get single band coherence and interferogram before application of the flattening phase

Get the Coherences

In [ ]:

coh_hh = single_baseline_single_pol_coh(
    primary_complex = hh_slc_pri,
    secondary_complex = hh_slc_sec,
    avg_kernel_shape = (5,5)
)

coh_xx = single_baseline_single_pol_coh(
    primary_complex = xx_slc_pri,
    secondary_complex = xx_slc_sec,
    avg_kernel_shape = (5,5)
)

coh_vv = single_baseline_single_pol_coh(
    primary_complex = vv_slc_pri,
    secondary_complex = vv_slc_sec,
    avg_kernel_shape = (5,5)
)

Get the Interferograms

In [ ]:
interferogram_hh = hh_slc_pri * np.conj(hh_slc_sec)

interferogram_xx = xx_slc_pri * np.conj(xx_slc_sec)

interferogram_vv = vv_slc_pri * np.conj(vv_slc_sec)

#### 4.2 Get single band coherence and interferogram after application of phases screens

generate the upsampled phases (see section 3)

In [ ]:
#####get the lut axis
relativeAzimuthTime_pri = product_primary.relativeAzimuthTime
slantRangeTime_pri      = product_primary.slantRangeTime
lut_az_axes_pri = (relativeAzimuthTime_pri - relativeAzimuthTime_pri[0]).astype(np.float64)


lut_range_axis_pri = slantRangeTime_pri - slantRangeTime_pri[0]

#####parse the main annotation


main_ann_primary = ET.parse(path_main_ann_primary)
main_ann_primary_root = main_ann_primary.getroot()

####get the data from the main annotations (primary and coregistered slc (L1C))


sarImage_pri = main_ann_primary_root.findall('sarImage')
#range coreg
firstSampleSlantRangeTime_pri = np.float64(sarImage_pri[0].findall("firstSampleSlantRangeTime")[0].text)
rangeTimeInterval_pri = np.float64(sarImage_pri[0].findall("rangeTimeInterval")[0].text)
numberOfSamples_pri = np.int64(sarImage_pri[0].findall("numberOfSamples")[0].text)

#az coreg
azimuthTimeInterval_pri = np.float64(sarImage_pri[0].findall("azimuthTimeInterval")[0].text)
firstLineAzimuthTime_pri = np.datetime64(sarImage_pri[0].findall("firstLineAzimuthTime")[0].text)
numberOfLines_pri = np.int64(sarImage_pri[0].findall("numberOfLines")[0].text)


######################## as per STA_P
#primary az axis
axis = 0
roi_pri =  [0, 0,numberOfLines_pri, numberOfSamples_pri]
time_step_pri = azimuthTimeInterval_pri
time_start_pri = 0
az_slc_axis_pri_stac = np.arange(roi_pri[axis + 2], dtype=np.float64) * time_step_pri

#pri rg axis
axis = 1
roi_pri =  [0, 0,numberOfLines_pri, numberOfSamples_pri]
time_step_pri = rangeTimeInterval_pri
time_start_pri = firstSampleSlantRangeTime_pri
range_slc_axis_pri_stac = np.arange(roi_pri[axis + 2], dtype=np.float64) * time_step_pri

# Print result clearly
print("\n[INFO] SLC Primary Range Axis:")
print(f"  - Number of samples : {numberOfSamples_pri}")
print(f"  - Range spacing     : {rangeTimeInterval_pri} seconds")
print(f"  - Max range axis    : {np.max(range_slc_axis_pri_stac):.6f} seconds")


In [ ]:
#get the flatteningPhasesScreen
flatteningPhaseScreen_co = product_secondary.coregistration_flatteningPhaseScreen
flatteningPhaseScreen_co = np.ma.masked_equal(flatteningPhaseScreen_co,noDataValue)

flatteningPhaseScreen_pri = product_primary.coregistration_flatteningPhaseScreen
flatteningPhaseScreen_pri = np.ma.masked_equal(flatteningPhaseScreen_pri, noDataValue)

#get the skpCalibrationPhaseScreen
skpCalibrationPhaseScreen_co = product_secondary.skpPhaseCalibration_skpCalibrationPhaseScreen
skpCalibrationPhaseScreen_co = np.ma.masked_equal(skpCalibrationPhaseScreen_co,noDataValue)

skpCalibrationPhaseScreen_pri = product_primary.skpPhaseCalibration_skpCalibrationPhaseScreen
skpCalibrationPhaseScreen_pri = np.ma.masked_equal(skpCalibrationPhaseScreen_pri, noDataValue)


In [ ]:
flatteningPhaseScreen_avg = np.mean(flatteningPhaseScreen_pri)
flatteningPhaseScreen_std = np.std(flatteningPhaseScreen_pri)

skpCalibrationPhaseScreen_avg = np.mean(skpCalibrationPhaseScreen_pri)
skpCalibrationPhaseScreen_std = np.mean(skpCalibrationPhaseScreen_pri)


std_multiplier = 3
aspect_ratio =0.2
colorbar_fraction = 0.05
colorbar_pad = 0.04

plt.close('all')
fig, axes = plt.subplots(1, 2, figsize=(10, 5)) 

# Primo plot: flatteningPhaseScreen
im1 = axes[0].imshow(
    flatteningPhaseScreen_pri,
    cmap="rainbow",
    aspect=aspect_ratio,
    interpolation="none",
)
axes[0].set_title('flatteningPhaseScreen_pri [rad]')
fig.colorbar(im1, ax=axes[0], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)

# Secondo plot: skpCalibrationPhaseScreen
im2 = axes[1].imshow(
    skpCalibrationPhaseScreen_pri,
    cmap="rainbow",
    aspect=aspect_ratio,
    interpolation="none",
)
axes[1].set_title('skpCalibrationPhaseScreen_pri [rad]')
fig.colorbar(im2, ax=axes[1], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)

fig.tight_layout()
plt.show()

In [ ]:
flatteningPhaseScreen_avg = np.mean(flatteningPhaseScreen_co)
flatteningPhaseScreen_std = np.std(flatteningPhaseScreen_co)

skpCalibrationPhaseScreen_avg = np.mean(skpCalibrationPhaseScreen_co)
skpCalibrationPhaseScreen_std = np.mean(skpCalibrationPhaseScreen_co)


std_multiplier = 3
aspect_ratio =0.2
colorbar_fraction = 0.05
colorbar_pad = 0.04

plt.close('all')
fig, axes = plt.subplots(1, 2, figsize=(10, 5)) 

# Primo plot: flatteningPhaseScreen
im1 = axes[0].imshow(
    flatteningPhaseScreen_co,
    cmap="rainbow",
    aspect=aspect_ratio,
    interpolation="none",
)
axes[0].set_title('flatteningPhaseScreen_co [rad]')
fig.colorbar(im1, ax=axes[0], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)

# Secondo plot: skpCalibrationPhaseScreen
im2 = axes[1].imshow(
    skpCalibrationPhaseScreen_co,
    cmap="rainbow",
    aspect=aspect_ratio,
    interpolation="none",
)
axes[1].set_title('skpCalibrationPhaseScreen_co [rad]')
fig.colorbar(im2, ax=axes[1], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)

fig.tight_layout()
plt.show()

In [ ]:
#preparation of the interpolators
axes_in = (lut_az_axes_pri, lut_range_axis_pri) # as showed before you can use also the *co axes since they are equal as expteced 
axes_out = (az_slc_axis_pri_stac, range_slc_axis_pri_stac) # as showed before you can use also the *co axes since they are equal as expteced 
degree_x = 1
degree_y = 1
smoother = 0.0
bbox=[
            min(np.min(axes_in[0]), np.min(axes_out[0])),
            max(np.max(axes_in[0]), np.max(axes_out[0])),
            max(np.min(axes_in[1]), np.min(axes_out[1])),
            max(np.max(axes_in[1]), np.max(axes_out[1])),
        ]
#interpolator for the coregistered data
flatteningPhaseScreen_co_interpolator = sp.interpolate.RectBivariateSpline(    
        axes_in[0],
        axes_in[1],
        flatteningPhaseScreen_co,
        bbox= bbox,
    kx=degree_x,
        ky=degree_y,
        s=smoother
     )
#interpolator for the primary data
flatteningPhaseScreen_pri_interpolator = sp.interpolate.RectBivariateSpline(    
        axes_in[0],
        axes_in[1],
        flatteningPhaseScreen_pri,
        bbox= bbox,
    kx=degree_x,
        ky=degree_y,
        s=smoother
     )

flatteningPhaseScreen_co_upsampled = flatteningPhaseScreen_co_interpolator(axes_out[0], axes_out[1])
flatteningPhaseScreen_pri_upsampled = flatteningPhaseScreen_pri_interpolator(axes_out[0], axes_out[1])


In [ ]:
# --- upsample SKP come exp(j·skp) ---
def upsample_phase_via_complex(phase_in: np.ndarray,
                               axes_in: tuple[np.ndarray, np.ndarray],
                               axes_out: tuple[np.ndarray, np.ndarray],
                               bbox: list[float],
                               kx: int = 1, ky: int = 1, s: float = 0.0,
                               nodata: float = -9999.0) -> np.ndarray:
    """
     Upsampling of a phase (in radians) using interpolation on cosine and sine, 
     to avoid artifacts caused by phase wrapping.
    """

    # Convert to real and imaginary parts of e^(j*phase)
    real_part = np.cos(phase_in)
    imag_part = np.sin(phase_in)

    interp_re = sp.interpolate.RectBivariateSpline(
        axes_in[0], axes_in[1], real_part,
        bbox=bbox, kx=kx, ky=ky, s=s
    )
    interp_im = sp.interpolate.RectBivariateSpline(
        axes_in[0], axes_in[1], imag_part,
        bbox=bbox, kx=kx, ky=ky, s=s
    )

    re_up = interp_re(axes_out[0], axes_out[1])
    im_up = interp_im(axes_out[0], axes_out[1])

    # Reconstruct the complex number e^(jφ) and extract its angle
    comp = re_up + 1j * im_up
    phase_up = np.angle(comp)

    return phase_up

skpCalibrationPhaseScreen_co_upsampled = upsample_phase_via_complex(
        skpCalibrationPhaseScreen_co,
        axes_in=(lut_az_axes_pri, lut_range_axis_pri),
        axes_out=(az_slc_axis_pri_stac, range_slc_axis_pri_stac),
        bbox=bbox, kx=degree_x, ky=degree_y, s=smoother, nodata=-9999.0
        )

skpCalibrationPhaseScreen_pri_upsampled = upsample_phase_via_complex(
        skpCalibrationPhaseScreen_pri,
        axes_in=(lut_az_axes_pri, lut_range_axis_pri),
        axes_out=(az_slc_axis_pri_stac, range_slc_axis_pri_stac),
        bbox=bbox, kx=degree_x, ky=degree_y, s=smoother, nodata=-9999.0
        )

In [ ]:
flatteningPhaseScreen_avg = np.nanmean(flatteningPhaseScreen_pri_upsampled)
flatteningPhaseScreen_std = np.nanstd(flatteningPhaseScreen_pri_upsampled)

skpCalibrationPhaseScreen_avg = np.mean(skpCalibrationPhaseScreen_pri_upsampled)
skpCalibrationPhaseScreen_std = np.mean(skpCalibrationPhaseScreen_pri_upsampled)


nan_value = 255
std_multiplier = 3
aspect_ratio =0.2
colorbar_fraction = 0.05
colorbar_pad = 0.04

plt.close('all')
fig, ax = plt.subplots(1, 2, figsize=(10, 5)) 
ax1 = ax[0].imshow(
    flatteningPhaseScreen_pri_upsampled, 
    cmap="rainbow",
    aspect=aspect_ratio, 
    interpolation="none",
    vmin =  flatteningPhaseScreen_avg - std_multiplier* flatteningPhaseScreen_std,
    vmax =  flatteningPhaseScreen_avg + std_multiplier* flatteningPhaseScreen_std
)
ax[0].set_title('flatteningPhaseScreen_pri_upsampled [rad]')
fig.colorbar(ax1, ax=ax[0], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)


ax2 = ax[1].imshow(
    skpCalibrationPhaseScreen_pri_upsampled, 
    cmap="rainbow",
    aspect=aspect_ratio, 
    interpolation="none",
    vmin =  skpCalibrationPhaseScreen_avg - std_multiplier* skpCalibrationPhaseScreen_std,
    vmax =  skpCalibrationPhaseScreen_avg + std_multiplier* skpCalibrationPhaseScreen_std
)
ax[1].set_title('skpCalibrationPhaseScreen_pri_upsampled [rad]')
fig.colorbar(ax2, ax=ax[1], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)

fig.tight_layout()
plt.show()


In [ ]:
flatteningPhaseScreen_avg = np.nanmean(flatteningPhaseScreen_co_upsampled)
flatteningPhaseScreen_std = np.nanstd(flatteningPhaseScreen_co_upsampled)

skpCalibrationPhaseScreen_avg = np.mean(skpCalibrationPhaseScreen_co_upsampled)
skpCalibrationPhaseScreen_std = np.mean(skpCalibrationPhaseScreen_co_upsampled)


nan_value = 255
std_multiplier = 3
aspect_ratio =0.2
colorbar_fraction = 0.05
colorbar_pad = 0.04

plt.close('all')
fig, ax = plt.subplots(1, 2, figsize=(10, 5)) 
ax1 = ax[0].imshow(
    flatteningPhaseScreen_co_upsampled, 
    cmap="rainbow",
    aspect=aspect_ratio, 
    interpolation="none",
    vmin =  flatteningPhaseScreen_avg - std_multiplier* flatteningPhaseScreen_std,
    vmax =  flatteningPhaseScreen_avg + std_multiplier* flatteningPhaseScreen_std
)
ax[0].set_title('flatteningPhaseScreen_co_upsampled [rad]')
fig.colorbar(ax1, ax=ax[0], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)


ax2 = ax[1].imshow(
    skpCalibrationPhaseScreen_co_upsampled, 
    cmap="rainbow",
    aspect=aspect_ratio, 
    interpolation="none",
    vmin =  skpCalibrationPhaseScreen_avg - std_multiplier* skpCalibrationPhaseScreen_std,
    vmax =  skpCalibrationPhaseScreen_avg + std_multiplier* skpCalibrationPhaseScreen_std
)
ax[1].set_title('skpCalibrationPhaseScreen_co_upsampled [rad]')
fig.colorbar(ax2, ax=ax[1], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)

fig.tight_layout()
plt.show()

application of the phases (flattening / skp)

In [ ]:
# Build total per-image corrections 'corr_pri' and 'corr_co'
    
flatteningPhaseScreen_pri_upsampled.min()
flatteningPhaseScreen_pri_upsampled.max()

corr_pri = 0.0
corr_co  = 0.0
    
if flatten == "None":
        # processor did NOT apply any screen -> we must compensate (flattening + skp)
        corr_pri = flatteningPhaseScreen_pri_upsampled + skpCalibrationPhaseScreen_pri_upsampled
        corr_co  = flatteningPhaseScreen_co_upsampled  + skpCalibrationPhaseScreen_co_upsampled
        print("[INFO] phaseCorrection: None -> applying (flattening + skp)")
        
elif flatten == "geometry":
        # processor applied only the geometry/DSI screen -> we add SKP only)
        corr_pri = skpCalibrationPhaseScreen_pri_upsampled
        corr_co  = skpCalibrationPhaseScreen_co_upsampled
        print("[INFO] phaseCorrection: Flattening-only -> applying SKP only")
        
elif flatten == "skp":
        # processor applied the full screen -> no correction here
        corr_pri = 0.0
        corr_co  = 0.0
        print("[INFO] phaseCorrection: Ground Phase (full) -> no additional correction")
else:
        raise ValueError(f"Unknown flatten option: {flatten}") 

In [ ]:
coh_hh_flat = single_baseline_single_pol_coh(
    primary_complex = hh_slc_pri,
    secondary_complex = hh_slc_sec * np.exp(1j * (corr_co - corr_pri)),
    avg_kernel_shape = (5,5)
)

coh_xx_flat = single_baseline_single_pol_coh(
    primary_complex = xx_slc_pri,
    secondary_complex = xx_slc_sec * np.exp(1j * (corr_co - corr_pri)),
    avg_kernel_shape = (5,5)
)

coh_vv_flat = single_baseline_single_pol_coh(
    primary_complex = vv_slc_pri,
    secondary_complex = vv_slc_sec * np.exp(1j * (corr_co - corr_pri)),
    avg_kernel_shape = (5,5)
)

In [ ]:
interferogram_hh_flat = ( hh_slc_pri* np.exp(1j * corr_pri)) * (np.conj(hh_slc_sec*np.exp(1j * corr_co)))
interferogram_xx_flat = ( xx_slc_pri* np.exp(1j * corr_pri)) * (np.conj(xx_slc_sec*np.exp(1j * corr_co)))
interferogram_vv_flat = ( vv_slc_pri* np.exp(1j * corr_pri)) * (np.conj(vv_slc_sec*np.exp(1j * corr_co)))

#### 4.4 Data plots: Interferograms

Get the phases

In [ ]:
interferogram_hh_deg = np.angle(interferogram_hh, deg=True)
interferogram_xx_deg = np.angle(interferogram_xx, deg=True)
interferogram_vv_deg = np.angle(interferogram_vv, deg=True)

interferogram_hh_flat_deg = np.angle(interferogram_hh_flat, deg=True)
interferogram_xx_flat_deg = np.angle(interferogram_xx_flat, deg=True)
interferogram_vv_flat_deg = np.angle(interferogram_vv_flat, deg=True)

Plot deg

In [ ]:
interferogram_hh_avg = np.mean(interferogram_hh_deg)
interferogram_hh_std = np.std(interferogram_hh_deg)

interferogram_xx_avg = np.mean(interferogram_xx_deg)
interferogram_xx_std = np.std(interferogram_xx_deg)

interferogram_vv_avg = np.mean(interferogram_vv_deg)
interferogram_vv_std = np.std(interferogram_vv_deg)

interferogram_hh_flat_avg = np.mean(interferogram_hh_flat_deg)
interferogram_hh_flat_std = np.std(interferogram_hh_flat_deg)

interferogram_xx_flat_avg = np.mean(interferogram_xx_flat_deg)
interferogram_xx_flat_std = np.std(interferogram_xx_flat_deg)

interferogram_vv_flat_avg = np.mean(interferogram_vv_flat_deg)
interferogram_vv_flat_std = np.std(interferogram_vv_flat_deg)

std_multiplier = 3 
aspect_ratio = 0.2
colorbar_fraction = 0.05
colorbar_pad = 0.04
interpolation = "nearest"

plt.close('all')
fig, axes = plt.subplots(2,3, figsize=(8,12)) 

ax1 = axes[0,0].imshow( 
    interferogram_hh_deg, 
    cmap='rainbow',
    aspect=aspect_ratio,
    vmin =  interferogram_hh_avg - std_multiplier* interferogram_hh_std,
    vmax =  interferogram_hh_avg + std_multiplier* interferogram_hh_std,
    interpolation=interpolation
)
axes[0,0].set_title('interferogram_hh_deg \n [deg]')


ax2 = axes[0,1].imshow( 
    interferogram_xx_deg, 
    cmap='rainbow',
    aspect=aspect_ratio,
    vmin =  interferogram_xx_avg - std_multiplier* interferogram_xx_std,
    vmax =  interferogram_xx_avg + std_multiplier* interferogram_xx_std,
    interpolation=interpolation
)
axes[0,1].set_title('interferogram_xx_deg \n [deg]')
axes[0,1].yaxis.set_tick_params(labelleft=False)


ax3 = axes[0,2].imshow( 
    interferogram_vv_deg, 
    cmap='rainbow',
    aspect=aspect_ratio,
    vmin =  interferogram_vv_avg - std_multiplier* interferogram_vv_std,
    vmax =  interferogram_vv_avg + std_multiplier* interferogram_vv_std,
    interpolation=interpolation
    
)
axes[0,2].set_title('interferogram_vv_deg \n [deg]')
axes[0,2].yaxis.set_tick_params(labelleft=False)

ax4 = axes[1,0].imshow( 
    interferogram_hh_flat_deg, 
    cmap='rainbow',
    aspect=aspect_ratio,
    vmin =  interferogram_hh_flat_avg - std_multiplier* interferogram_hh_flat_std,
    vmax =  interferogram_hh_flat_avg + std_multiplier* interferogram_hh_flat_std,
    interpolation=interpolation
)
axes[1,0].set_title('interferogram_hh_flat_deg \n [deg]')


ax5 = axes[1,1].imshow( 
    interferogram_xx_flat_deg, 
    cmap='rainbow',
    aspect=aspect_ratio,
    vmin =  interferogram_xx_flat_avg - std_multiplier* interferogram_xx_flat_std,
    vmax =  interferogram_xx_flat_avg + std_multiplier* interferogram_xx_flat_std,
    interpolation=interpolation
)
axes[1,1].set_title('interferogram_xx_flat_deg \n [deg]')
axes[1,1].yaxis.set_tick_params(labelleft=False)


ax6 = axes[1,2].imshow( 
    interferogram_vv_flat_deg, 
    cmap='rainbow',
    aspect=aspect_ratio,
    vmin =  interferogram_vv_flat_avg - std_multiplier* interferogram_vv_flat_std,
    vmax =  interferogram_vv_flat_avg + std_multiplier* interferogram_vv_flat_std,
    interpolation=interpolation
    
)
axes[1,2].set_title('interferogram_vv_flat_deg \n [deg]')
axes[1,2].yaxis.set_tick_params(labelleft=False)


fig.colorbar(ax1, ax=axes[0,0], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.colorbar(ax2, ax=axes[0,1], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.colorbar(ax3, ax=axes[0,2], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)   


fig.colorbar(ax4, ax=axes[1,0], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.colorbar(ax5, ax=axes[1,1], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.colorbar(ax6, ax=axes[1,2], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)        

fig.tight_layout()
plt.show()

In [ ]:
# Plot histograms
bin = 40

shape = np.shape(interferogram_hh_deg)
plt.close('all')
fig, axes = plt.subplots(2, 3, figsize=(12, 8))

axes[0, 0].hist(interferogram_hh_deg.reshape(shape[0]*shape[1]), bins=bin,  alpha=0.7)
axes[0, 0].set_title('interferogram_hh_deg [deg]')

axes[0, 1].hist(interferogram_xx_deg.reshape(shape[0]*shape[1]), bins=bin,  alpha=0.7)
axes[0, 1].set_title('interferogram_xx_deg [deg]')
axes[0, 1].yaxis.set_tick_params(labelleft=False)

axes[0, 2].hist(interferogram_vv_deg.reshape(shape[0]*shape[1]), bins=bin,  alpha=0.7)
axes[0, 2].set_title('interferogram_vv_deg [deg]')
axes[0, 2].yaxis.set_tick_params(labelleft=False)

axes[1, 0].hist(interferogram_hh_flat_deg.reshape(shape[0]*shape[1]), bins=bin,  alpha=0.7)
axes[1, 0].set_title('interferogram_hh_flat_deg [deg]')

axes[1, 1].hist(interferogram_xx_flat_deg.reshape(shape[0]*shape[1]), bins=bin,  alpha=0.7)
axes[1, 1].set_title('interferogram_xx_flat_deg [deg]')
axes[1, 1].yaxis.set_tick_params(labelleft=False)

axes[1, 2].hist(interferogram_vv_flat_deg.reshape(shape[0]*shape[1]), bins=bin,  alpha=0.7)
axes[1, 2].set_title('interferogram_vv_flat_deg [deg]')
axes[1, 2].yaxis.set_tick_params(labelleft=False)

fig.tight_layout()
plt.show()

#### 4.5 Data plots Coh

Get the amplitidues and the phases

In [ ]:
coh_hh_deg = np.angle(coh_hh, deg=True)
coh_xx_deg = np.angle(coh_xx, deg=True)
coh_vv_deg = np.angle(coh_vv, deg=True)

coh_hh_amp = np.abs(coh_hh)
coh_xx_amp = np.abs(coh_xx)
coh_vv_amp = np.abs(coh_vv)


coh_hh_flat_deg = np.angle(coh_hh_flat, deg=True)
coh_xx_flat_deg = np.angle(coh_xx_flat, deg=True)
coh_vv_flat_deg = np.angle(coh_vv_flat, deg=True)

coh_hh_flat_amp = np.abs(coh_hh_flat)
coh_xx_flat_amp = np.abs(coh_xx_flat)
coh_vv_flat_amp = np.abs(coh_vv_flat)

Plot phases deg

In [ ]:
coh_hh_avg = np.mean(coh_hh_deg)
coh_hh_std = np.std(coh_hh_deg)

coh_xx_avg = np.mean(coh_xx_deg)
coh_xx_std = np.std(coh_xx_deg)

coh_vv_avg = np.mean(coh_vv_deg)
coh_vv_std = np.std(coh_vv_deg)

coh_hh_flat_avg = np.mean(coh_hh_flat_deg)
coh_hh_flat_std = np.std(coh_hh_flat_deg)

coh_xx_flat_avg = np.mean(coh_xx_flat_deg)
coh_xx_flat_std = np.std(coh_xx_flat_deg)

coh_vv_flat_avg = np.mean(coh_vv_flat_deg)
coh_vv_flat_std = np.std(coh_vv_flat_deg)

std_multiplier = 3 
aspect_ratio = 0.2
colorbar_fraction = 0.05
colorbar_pad = 0.04
interpolation = "nearest"

plt.close('all')
fig, axes = plt.subplots(2,3, figsize=(8,12)) 

ax1 = axes[0,0].imshow( 
    coh_hh_deg, 
    cmap='rainbow',
    aspect=aspect_ratio,
    vmin =  coh_hh_avg - std_multiplier* coh_hh_std,
    vmax =  coh_hh_avg + std_multiplier* coh_hh_std,
    interpolation=interpolation
)
axes[0,0].set_title('coh_hh_deg [deg]')


ax2 = axes[0,1].imshow( 
    coh_xx_deg, 
    cmap='rainbow',
    aspect=aspect_ratio,
    vmin =  coh_xx_avg - std_multiplier* coh_xx_std,
    vmax =  coh_xx_avg + std_multiplier* coh_xx_std,
    interpolation=interpolation
)
axes[0,1].set_title('coh_xx_deg [deg]')
axes[0,1].yaxis.set_tick_params(labelleft=False)


ax3 = axes[0,2].imshow( 
    coh_vv_deg, 
    cmap='rainbow',
    aspect=aspect_ratio,
    vmin =  coh_vv_avg - std_multiplier* coh_vv_std,
    vmax =  coh_vv_avg + std_multiplier* coh_vv_std,
    interpolation=interpolation
    
)
axes[0,2].set_title('coh_vv_deg [deg]')
axes[0,2].yaxis.set_tick_params(labelleft=False)

ax4 = axes[1,0].imshow( 
    coh_hh_flat_deg, 
    cmap='rainbow',
    aspect=aspect_ratio,
    vmin =  coh_hh_flat_avg - std_multiplier* coh_hh_flat_std,
    vmax =  coh_hh_flat_avg + std_multiplier* coh_hh_flat_std,
    interpolation=interpolation
)
axes[1,0].set_title('coh_hh_flat_deg [deg]')


ax5 = axes[1,1].imshow( 
    coh_xx_flat_deg, 
    cmap='rainbow',
    aspect=aspect_ratio,
    vmin =  coh_xx_flat_avg - std_multiplier* coh_xx_flat_std,
    vmax =  coh_xx_flat_avg + std_multiplier* coh_xx_flat_std,
    interpolation=interpolation
)
axes[1,1].set_title('coh_xx_flat_deg [deg]')
axes[1,1].yaxis.set_tick_params(labelleft=False)


ax6 = axes[1,2].imshow( 
    coh_vv_flat_deg, 
    cmap='rainbow',
    aspect=aspect_ratio,
    vmin =  coh_vv_flat_avg - std_multiplier* coh_vv_flat_std,
    vmax =  coh_vv_flat_avg + std_multiplier* coh_vv_flat_std,
    interpolation=interpolation
    
)
axes[1,2].set_title('coh_vv_flat_deg [deg]')
axes[1,2].yaxis.set_tick_params(labelleft=False)


fig.colorbar(ax1, ax=axes[0,0], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.colorbar(ax2, ax=axes[0,1], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.colorbar(ax3, ax=axes[0,2], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)   


fig.colorbar(ax4, ax=axes[1,0], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.colorbar(ax5, ax=axes[1,1], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.colorbar(ax6, ax=axes[1,2], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)        

fig.tight_layout()
plt.show()

In [ ]:
# Plot histograms
bin = 100


shape = np.shape(coh_hh_deg)
plt.close('all')
fig, axes = plt.subplots(2, 3, figsize=(12, 8))

axes[0, 0].hist(coh_hh_deg.reshape(shape[0]*shape[1]), bins=bin,  alpha=0.7)
axes[0, 0].set_title('coh_hh_deg [deg]')

axes[0, 1].hist(coh_xx_deg.reshape(shape[0]*shape[1]), bins=bin,  alpha=0.7)
axes[0, 1].set_title('coh_xx_deg [deg]')
axes[0, 1].yaxis.set_tick_params(labelleft=False)

axes[0, 2].hist(coh_vv_deg.reshape(shape[0]*shape[1]), bins=bin,  alpha=0.7)
axes[0, 2].set_title('coh_vv_deg [deg]')
axes[0, 2].yaxis.set_tick_params(labelleft=False)

axes[1, 0].hist(coh_hh_flat_deg.reshape(shape[0]*shape[1]), bins=bin,  alpha=0.7)
axes[1, 0].set_title('coh_hh_flat_deg [deg]')

axes[1, 1].hist(coh_xx_flat_deg.reshape(shape[0]*shape[1]), bins=bin,  alpha=0.7)
axes[1, 1].set_title('coh_xx_flat_deg [deg]')
axes[1, 1].yaxis.set_tick_params(labelleft=False)

axes[1, 2].hist(coh_vv_flat_deg.reshape(shape[0]*shape[1]), bins=bin,  alpha=0.7)
axes[1, 2].set_title('coh_vv_flat_deg [deg]')
axes[1, 2].yaxis.set_tick_params(labelleft=False)

fig.tight_layout()
plt.show()


Plot amp

In [ ]:
coh_hh_avg = np.mean(coh_hh_amp)
coh_hh_std = np.std(coh_hh_amp)

coh_xx_avg = np.mean(coh_xx_amp)
coh_xx_std = np.std(coh_xx_amp)

coh_vv_avg = np.mean(coh_vv_amp)
coh_vv_std = np.std(coh_vv_amp)

coh_hh_flat_avg = np.mean(coh_hh_flat_amp)
coh_hh_flat_std = np.std(coh_hh_flat_amp)

coh_xx_flat_avg = np.mean(coh_xx_flat_amp)
coh_xx_flat_std = np.std(coh_xx_flat_amp)

coh_vv_flat_avg = np.mean(coh_vv_flat_amp)
coh_vv_flat_std = np.std(coh_vv_flat_amp)

std_multiplier = 1 
aspect_ratio = 0.2
colorbar_fraction = 0.05
colorbar_pad = 0.04
interpolation = "nearest"
plt.close('all')
fig, axes = plt.subplots(2,3, figsize=(8,12)) 

ax1 = axes[0,0].imshow( 
    coh_hh_amp, 
    cmap='rainbow',
    aspect=aspect_ratio,
    vmin =  0,
    vmax =  1,
    interpolation=interpolation
)
axes[0,0].set_title('coh_hh_amp [amp]')


ax2 = axes[0,1].imshow( 
    coh_xx_amp, 
    cmap='rainbow',
    aspect=aspect_ratio,
    vmin =  0,
    vmax =  1,
    interpolation=interpolation
)
axes[0,1].set_title('coh_xx_amp [amp]')
axes[0,1].yaxis.set_tick_params(labelleft=False)


ax3 = axes[0,2].imshow( 
    coh_vv_amp, 
    cmap='rainbow',
    aspect=aspect_ratio,
    vmin =  0,
    vmax =  1,
    interpolation=interpolation
    
)
axes[0,2].set_title('coh_vv_amp [amp]')
axes[0,2].yaxis.set_tick_params(labelleft=False)

ax4 = axes[1,0].imshow( 
    coh_hh_flat_amp, 
    cmap='rainbow',
    aspect=aspect_ratio,
    vmin =  0,
    vmax =  1,
    interpolation=interpolation
)
axes[1,0].set_title('coh_hh_flat_amp [amp]')


ax5 = axes[1,1].imshow( 
    coh_xx_flat_amp, 
    cmap='rainbow',
    aspect=aspect_ratio,
    vmin =  0,
    vmax =  1,
    interpolation=interpolation
)
axes[1,1].set_title('coh_xx_flat_amp [amp]')
axes[1,1].yaxis.set_tick_params(labelleft=False)


ax6 = axes[1,2].imshow( 
    coh_vv_flat_amp, 
    cmap='rainbow',
    aspect=aspect_ratio,
    vmin =  0,
    vmax =  1,
    interpolation=interpolation
    
)
axes[1,2].set_title('coh_vv_flat_amp [amp] \n with screens')
axes[1,2].yaxis.set_tick_params(labelleft=False)


fig.colorbar(ax1, ax=axes[0,0], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.colorbar(ax2, ax=axes[0,1], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.colorbar(ax3, ax=axes[0,2], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)   


fig.colorbar(ax4, ax=axes[1,0], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.colorbar(ax5, ax=axes[1,1], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.colorbar(ax6, ax=axes[1,2], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)        

fig.tight_layout()
plt.show()

In [ ]:
# Plot histograms
bin = 100


shape = np.shape(coh_hh_amp)
plt.close('all')
fig, axes = plt.subplots(2, 3, figsize=(12, 8))

axes[0, 0].hist(coh_hh_amp.reshape(shape[0]*shape[1]), bins=bin,  alpha=0.7)
axes[0, 0].set_title('coh_hh_amp [amp]')

axes[0, 1].hist(coh_xx_amp.reshape(shape[0]*shape[1]), bins=bin,  alpha=0.7)
axes[0, 1].set_title('coh_xx_amp [amp]')
axes[0, 1].yaxis.set_tick_params(labelleft=False)

axes[0, 2].hist(coh_vv_amp.reshape(shape[0]*shape[1]), bins=bin,  alpha=0.7)
axes[0, 2].set_title('coh_vv_amp [amp]')
axes[0, 2].yaxis.set_tick_params(labelleft=False)

axes[1, 0].hist(coh_hh_flat_amp.reshape(shape[0]*shape[1]), bins=bin,  alpha=0.7)
axes[1, 0].set_title('coh_hh_flat_amp [amp]')

axes[1, 1].hist(coh_xx_flat_amp.reshape(shape[0]*shape[1]), bins=bin,  alpha=0.7)
axes[1, 1].set_title('coh_xx_flat_amp [amp]')
axes[1, 1].yaxis.set_tick_params(labelleft=False)

axes[1, 2].hist(coh_vv_flat_amp.reshape(shape[0]*shape[1]), bins=bin,  alpha=0.7)
axes[1, 2].set_title('coh_vv_flat_amp [amp]')
axes[1, 2].yaxis.set_tick_params(labelleft=False)

fig.tight_layout()
plt.show()